In [7]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys

#
from tqdm import tqdm, trange

#
import opexebo as op

import astropy
from astropy.convolution import Gaussian2DKernel



#
from utils import load_locs_traces_running, check_cell_id_field


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_467576/44542414.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
#############################################################
################### DEFAULT LOADING OF DATA #################
#############################################################

#fname_locs = ('/media/cat/4TB1/donato/nathalie/DON-007050/FS1/DON-007050_20211022_'+
#         'TR-BSL_FS1-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000_locs.npy')
fname_locs = ('/media/cat/4TB/donato/nathalie/DON-007050/FS9/DON-007050_20211030_TR-BSL_FS9-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000_locs.npy')

#
#fname_traces = '/media/cat/4TB1/donato/nathalie/DON-007050/FS1/binarized_traces.npz'
fname_traces = '/media/cat/4TB/donato/nathalie/DON-007050/FS9/binarized_traces.npz'

#
arena_x = [300,1550]
arena_y = [175,1400]
arena_size = [80,80]
arena_shape = 'square'
bin_width = 2.5

#
locs, upphases, times, filtered_Fs = load_locs_traces_running(fname_locs,
                                                              fname_traces,
                                                              arena_size)

# compute spatial occpuancy map; only requires the locatoins and 
occ_map, coverage, bin_edges = op.analysis.spatial_occupancy(times, 
                                                             locs.T, 
                                                             arena_size, 
                                                             bin_width=bin_width)

fname_occ_map = os.path.join(os.path.split(fname_traces)[0],"occ_map.npy")
occ_map.dump(fname_occ_map)
                             

#
x_edges = bin_edges[0]
y_edges = bin_edges[1]

(36000, 2)
(35999,)
Locs:  (35110, 2)  uphases:  (365, 35110)


In [10]:
#############################################################
################### DEFAULT LOADING OF DATA #################
#############################################################

#
sigma = 1.5
plotting = False
circular_shuffle = False
n_tests = 100

#
#cell_ids = np.arange(0,100,1)
cell_ids = np.arange(0,upphases.shape[0],1)
#cell_ids = np.array([33])
#cell_ids = [18]
#cell_ids = np.arange(18,20,1)

#
overlaps = []
s_info = []
res_arrays = []
cohs_array = []
D_array = []

# make root directory for data saving
root_dir = os.path.join(os.path.split(fname_traces)[0],
                             "cell_analysis")
try:
    os.mkdir(root_dir)
except:
    pass

#
for cell_id in tqdm(cell_ids):

    #
    fname_out = os.path.join(root_dir, str(cell_id)+".npy")
    
    #
    if os.path.exists(fname_out):
        continue
    
    # get dictionary with all data
    D = check_cell_id_field(cell_id,
                            upphases,
                            filtered_Fs,
                            locs,
                            occ_map,
                            arena_size,
                            sigma,
                            circular_shuffle,
                            n_tests,
                            x_edges,
                            y_edges
                           )
       
    # save dictionary
    np.save(fname_out, D, allow_pickle=True)   




100%|███████████████████████████████████████████████████████████████████████████████████████▊| 364/365 [00:06<00:00, 52.59it/s]


IndexError: index 365 is out of bounds for axis 0 with size 365

In [14]:
fname = '/media/cat/4TB/donato/nathalie/DON-007050/FS9/cell_dictionary.npy'
data = np.load(fname, allow_pickle=True)
print (len(data))
print (data[0])

2
{'cell_id': 18, 'overlaps': [0.9583333333333334, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.2727272727272727, 0.0, 0.1505016722408027, 0.0, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.0, 0.0888030888030888, 0.15730337078651685, 0.08522727272727272, 0.34645669291

In [487]:
##################################################
############# VISUALIZE MULTI_CELL ###############
##################################################
#
cell_id = 6

#
print (len(D_array))
print (D_array[cell_id].keys())
print (D_array[cell_id]['overlaps_zscores'])

def plot_all_metrics(D_array):
    
    #
    sis = []
    sis_z = []
    overls = []
    overls_z = []
    cohs = []
    spars = []
    si_rates = []
    for k in range(len(D_array)):
        cell = D_array[k]
        si = cell['spatial_info'][0]
        si_z = cell['spatial_info_zscores'][0]
        overl = cell['overlaps'][0]
        overl_z = cell['overlaps_zscores'][0]
        #
        temp = cell['res_array']
        spar = []
        si_rate = []
        for k in range(len(temp)):
            spar.append(temp[k]['sparsity'])
            si_rate.append(temp[k]['spatial_information_rate'])    
        spar = np.nanmean(np.hstack((spar)))
        si_rate = np.nanmean(np.hstack((si_rate)))
        
        #
        coh = np.nanmean(np.hstack(cell['coherence']))

        
        #
        sis.append(si)
        sis_z.append(si_z)
        overls.append(overl)
        overls_z.append(overl_z)
        cohs.append(coh)
        spars.append(spar)
        si_rates.append(si_rate)

    
    plt.figure(figsize=(15,10))

    ############################
    ax = plt.subplot(2,3,1)
    color='blue'
    ax.scatter(overls,
                sis,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" spatial info (average 1st half and 2nd half)")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], sis[k]))
    
    ############################
    ax = plt.subplot(2,3,2)
    color='blue'
    print (overls_z)
    ax.scatter(overls,
                sis_z,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" zscore spatial info")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], sis_z[k]))
        
    ############################
    ax = plt.subplot(2,3,3)
    color='blue'
    print (overls_z)
    ax.scatter(overls,
                si_rates,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" spatial information rate")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], si_rates[k]))
        
        
        
    ############################
    ax = plt.subplot(2,3,4)
    color='red'
    ax.scatter(overls,
                overls_z,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" zscore overlap")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], overls_z[k]))
        
    ############################
    ax = plt.subplot(2,3,5)
    color='green'
    ax.scatter(overls,
                cohs,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" coherence")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], cohs[k]))

        
        
    ############################
    ax = plt.subplot(2,3,5)
    color='black'
    ax.scatter(overls,
                spars,
                s=100,
                #alpha=overl_z,
                edgecolor='black',
                c=color)
    plt.xlabel(" overlap of place field (1st half and 2nd half)")
    plt.ylabel(" sparsity")
    for k in range(len(overls)):
        ax.annotate(str(cell_ids[k]), (overls[k], spars[k]))
    
    
    
    plt.show()

plot_all_metrics(D_array)




10
dict_keys(['cell_id', 'overlaps', 'spatial_info', 'overlaps_zscores', 'spatial_info_zscores', 'coherence', 'fields_map_good', 'res_array', 'rms_good'])
[-0.82595826  1.58951752 -0.82595826 -0.82595826 -0.32273414  1.58951752
 -0.82595826 -0.82595826 -0.32273414  1.58951752 -0.82595826 -0.82595826
 -0.32273414  1.58951752 -0.82595826 -0.82595826 -0.32273414  1.58951752
 -0.82595826 -0.82595826 -0.32273414  1.58951752 -0.82595826 -0.82595826
 -0.32273414  1.58951752 -0.82595826 -0.82595826 -0.32273414  1.58951752
 -0.82595826 -0.82595826 -0.32273414 -0.82595826 -0.82595826  0.09978423
  0.11376413 -0.82595826 -0.82595826  0.09978423  0.11376413 -0.82595826
 -0.82595826  0.09978423  0.11376413 -0.82595826 -0.82595826  0.09978423
  0.11376413 -0.82595826 -0.82595826  0.09978423  0.11376413 -0.82595826
 -0.82595826  0.09978423  0.11376413 -0.82595826 -0.82595826  0.09978423
  0.11376413 -0.82595826 -0.82595826  0.09978423  0.11376413 -0.82595826
  1.64657601  1.50005546 -0.64914979 -0.82

In [472]:
cell_id = 23

for k in range(len(D_array)):
    cell = D_array[k]
    if cell['cell_id'] == cell_id:
        rms_good = cell['rms_good']
        fields_map_good = cell['fields_map_good']

plt.figure()
plot_fields(rms_good, fields_map_good)
plt.show()


In [325]:
#feature = 'spatial_information_rate'
feature = 'spatial_information_content'
#feature = 'sparsity'
#feature = 'selectivity'
#feature = 'peak_rate'
#feature = 'mean_rate'

#
s_info = []
for k in range(len(res_arrays)):
    temp1 = res_arrays[k][0][feature]
    temp2 = res_arrays[k][1][feature]
    #print (temp1, temp2)
    #print (np.mean(temp1,temp2))
    #
    s_info.append(np.nanmean(np.hstack((temp1,temp2))))


/tmp/ipykernel_394468/2958903581.py:16: RuntimeWarning: Mean of empty slice
  s_info.append(np.nanmean(np.hstack((temp1,temp2))))


In [328]:
feature = 'coherence'
s_info = []
for k in range(len(res_arrays)):
    temp1 = cohs_array[k][0]
    temp2 = cohs_array[k][1]
    #print (temp1, temp2)
    #print (np.mean(temp1,temp2))
    #
    s_info.append(np.nanmean(np.hstack((temp1,temp2))))


/tmp/ipykernel_394468/3649763955.py:9: RuntimeWarning: Mean of empty slice
  s_info.append(np.nanmean(np.hstack((temp1,temp2))))


In [329]:
color = 'blue'
if circular_shuffle:
    color='red'

plt.figure()
plt.scatter(overlaps,
            s_info,
            c=color)

plt.xlabel(" overlap of place field (1st half and 2nd half)")
plt.ylabel(feature + " (average 1st half and 2nd half)")
plt.suptitle("mouse 7050")
plt.show()

In [150]:
    plot_fields(rms, fields_map)

    if True:
        plt.show()
    else:
        plt.savefig("/home/cat/fields.svg")
        plt.close()

/tmp/ipykernel_394468/4117706899.py:672: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax=plt.subplot(2,2,k+1)


In [39]:
#print (res)
text = "SI_rate: "+ str(round(res['spatial_information_rate'],2)) + \
        "SI_cont: "+ str(round(res['spatial_information_content'],2)) + \
        "Sparse: "+ str(round(res['sparsity'],2)) + ' \n '+ \
        "Select: "+  str(round(res['selectivity'],2))+ \
        "Peak_r: "+  str(round(res['peak_rate'],2))+ \
        "Mean_r: "+  str(round(res['mean_rate'],2))

print (text)


['SI_rate: 2.65SI_cont: 1.14Sparse: 0.45 \n Select: 4.59Peak_r: 10.64Mean_r: 0.57']


In [9]:
###############################################################
############# SPATIAL SELETIVITY VS. SELECTIVITY ##############
###############################################################
scale_flag = False
scale_value = 1
sigma = 1.5

cell_ids = np.arange(upphases.shape[0])
#cell_ids = np.arange(103,200,1)
spc_array = []
sel_array = []
for cell_id in tqdm(cell_ids): 
    #print (cell_id)
    
    _, rms, _, _ = get_rms_and_place_field(cell_id,
                                           upphases.copy(),
                                           filtered_Fs.copy(),
                                           occ_map.copy(),
                                           arena_size.copy(),
                                           sigma,
                                           scale_flag,
                                           scale_value
                                           #limits
                                          )
    #
    res = op.analysis.rate_map_stats(rms.copy(), 
                                     occ_map.copy(), 
                                     debug=False)
    
   # print (res)
    spc = res['spatial_information_content']
    sel = res['selectivity']
    if np.isnan(spc) or np.isnan(sel):
        print ("ceel Id: ", spc, sel)

    spc_array.append(spc)
    sel_array.append(sel)
    
plt.figure()

plt.scatter(spc_array, sel_array)
plt.xlabel("special_information_content")
plt.ylabel("selectivity")

plt.show()



 22%|██▏       | 103/476 [00:09<00:29, 12.64it/s]

ceel Id:  nan nan


 37%|███▋      | 177/476 [00:16<00:27, 11.02it/s]


KeyboardInterrupt: 